In [1]:
import sys
# TO CHANGE
BASEDIR = "../../../.."
sys.path.insert(0, BASEDIR)

In [2]:
from pprint import pprint

In [3]:
from src.kg_model import KnowledgeGraphModel, KnowledgeGraphModelConfig
from src.db_drivers.vector_driver import EmbedderModelConfig
from src.pipelines.memorize import MemPipelineConfig, MemPipeline, LLMUpdatorConfig

from src.pipelines.qa.kg_reasoning.weak_reasoner import WeakKGReasonerConfig, WeakKGReasoner
from src.pipelines.qa.kg_reasoning.weak_reasoner import QALLMGeneratorConfig, QueryLLMParserConfig, KnowledgeComparatorConfig, KnowledgeRetrieverConfig

from src.pipelines.qa.kg_reasoning.medium_reasoner import MediumKGReasonerConfig, MediumKGReasoner
from src.pipelines.qa.kg_reasoning.medium_reasoner import AnswerGeneratorConfig, ClueAnswerGeneratorConfig, ClueAnswersSummarizerConfig, \
    ClueQueriesGeneratorConfig, EntitiesExtractorConfig, Entities2NodesMatcherConfig, SearchPlanEnhancerConfig

/home/dzigen/Desktop/Projects/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Пример настройки KGReasoner-стадии в рамках QA-пайплайна

1. Инициализация модели графа знаний

In [4]:
kg_config = KnowledgeGraphModelConfig(
    nodestree_config=None # Модель дерева вершин строиться не будет
)

EMBEDDER_MODEL_PATH = '../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL
kg_config.embedders_configs['m-e5-small'] = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
kg_model = KnowledgeGraphModel(kg_config)

No sentence-transformers model found with name ../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


2. Инициализация Memorize-пайплайна

In [5]:
mem_config = MemPipelineConfig(
    updator_config=LLMUpdatorConfig(
        delete_obsolete_info=False # Выключаем механизм по поиску/удалению устревших знаний в графе при добавлении новой информации
    )
)
mem_pipeline = MemPipeline(kg_model, mem_config)

In [6]:
kg_model.clear()

3. Добавление информации в граф знаний

In [7]:
TEXT_EXAMPLES = [
    "Sasha was walking along the highway.", 
    "Masha was walking along the highway.", 
    "The ship was sailing along the water canal.", 
    "The motorboat was sailing along the river."]

In [8]:
extracted_triplets = []
for example in TEXT_EXAMPLES:
    tmp_extracted_triplets, _ = mem_pipeline.remember(example)
    extracted_triplets += tmp_extracted_triplets

In [9]:
kg_model.count_items(detailed=True)

{'graph_info': {'triplets': {'simple': 5, 'hyper': 10, 'episodic': 16},
  'nodes': {'object': 10, 'hyper': 5, 'episodic': 4}},
 'embeddings_info': {'nodes': {'object': {'nodes_dense': 10,
    'nodes_sparse_bm25': 10},
   'hyper': {'nodes_dense': 5, 'nodes_sparse_bm25': 5},
   'episodic': {'nodes_dense': 4, 'nodes_sparse_bm25': 4}},
  'triplets': {'triplets_dense': 14, 'triplets_sparse_bm25': 14}},
 'nodestree_info': None}

In [10]:
kg_model.check_consistency()

True

In [11]:
print("mem tgen cache:")
pprint(mem_pipeline.get_agent_tgen_stat())
print("mem casual cache:")
pprint(mem_pipeline.get_cache_stat())

mem tgen cache:
{'extractor': {'thesises_extraction_solver': None,
               'triplets_extraction_solver': None},
 'updator': {'replace_hyper_solver': None, 'replace_simple_solver': None}}
mem casual cache:
{'MemPipeline': None,
 'extractor': {'LLMExtractor': None,
               'thesises_extraction_solver': None,
               'triplets_extraction_solver': None},
 'updator': {'LLMUpdator': None,
             'replace_hyper_solver': None,
             'replace_simple_solver': None}}


4.1. Инициализация 'weak' KGReasoner-стадии

In [14]:
weak_reasoner_config = WeakKGReasonerConfig(
    query_parser_config=QueryLLMParserConfig(lang='en'),
    knowledge_comparator_config=KnowledgeComparatorConfig(),
    knowledge_retriever_config=KnowledgeRetrieverConfig(),
    answer_generator_config=QALLMGeneratorConfig(lang='en')
)
pprint(weak_reasoner_config, depth=1, width=200)

WeakKGReasonerConfig(query_parser_config=QueryLLMParserConfig(lang='en',
                                                              agent_gen_stategy=None,
                                                              kw_extraction_task_config=AgentTaskSolverConfig(version='v2',
                                                                                                              suites={...},
                                                                                                              formate_context_func=<function kwe_custom_formate at 0x7f8f0cb3a8c0>,
                                                                                                              postprocess_answer_func=<function kwe_custom_postprocess at 0x7f8f0cb3a950>,
                                                                                                              cache_table_name='qa_agent_kwe_task_cache',
                                                                        

In [15]:
weak_reasoner = WeakKGReasoner(kg_model, weak_reasoner_config)

In [16]:
QUERY_EXAMPLES = ["Which of the following people walked along the highway: Sasha, Masha, Katya?",
                  "Have motorboat was ever sailed through a water canal?"]

In [17]:
for query in QUERY_EXAMPLES:
    print("\nQuery: ", query)
    answer, rinfo = weak_reasoner.perform(query)
    print('Return status: ')
    pprint(rinfo)
    print("Answer: ", answer)


Query:  Which of the following people walked along the highway: Sasha, Masha, Katya?
Return status: 
ReturnInfo(occurred_warning=[], status=<ReturnStatus.success: 0>, message='')
Answer:  Sasha

Query:  Have motorboat was ever sailed through a water canal?
Return status: 
ReturnInfo(occurred_warning=[], status=<ReturnStatus.success: 0>, message='')
Answer:  Yes


In [18]:
print("qa tgen cache:")
pprint(weak_reasoner.get_agent_tgen_stat())
print("qa casual cache:")
pprint(weak_reasoner.get_cache_stat())

qa tgen cache:
{'answer_generator': {'answer_generator_solver': None},
 'query_parser': {'kw_extraction_solver': None}}
qa casual cache:
{'WeakKGReasoner': None,
 'answer_generator': {'QALLMGenerator': None, 'answer_generator_solver': None},
 'knowledge_comparator': {'KnowledgeComparator': None},
 'knowledge_retriever': {'KnowledgeRetriever': None,
                         'triplets_filter': {'TripletsFilter': None},
                         'triplets_retriever': {'BeamSearchTripletsRetriever': {'BeamSearchTripletsRetriever': None,
                                                                                'traversal_cache': None}}},
 'query_parser': {'QueryLLMParser': None, 'kw_extraction_solver': None}}


4.2. Инициализация 'medium' KGReasoner-стадии

In [19]:
medium_reasoner_config = MediumKGReasonerConfig(
    searchplan_enhancer_config=SearchPlanEnhancerConfig(lang='en'),
    entities_extractor_config=EntitiesExtractorConfig(lang='en'),
    e2n_matcher_config=Entities2NodesMatcherConfig(max_n=1),
    cluequeries_generator_config=ClueQueriesGeneratorConfig(lang='en'),
    knowledge_retriever_config=KnowledgeRetrieverConfig(),
    clueanswer_generator_config=ClueAnswerGeneratorConfig(lang='en'),
    clueanswers_summarizer_config=ClueAnswersSummarizerConfig(lang='en'),
    answer_generator_config=AnswerGeneratorConfig(lang='en')
)
pprint(medium_reasoner_config, depth=1, width=200)

MediumKGReasonerConfig(searchplan_enhancer_config=SearchPlanEnhancerConfig(lang='en',
                                                                           agent_gen_stategy=None,
                                                                           plan_initing_agent_task_config=AgentTaskSolverConfig(version='v1',
                                                                                                                                suites={...},
                                                                                                                                formate_context_func=<function planinit_custom_formate at 0x7f8f0cbbd990>,
                                                                                                                                postprocess_answer_func=<function planinit_custom_postprocess at 0x7f8f0cbbda20>,
                                                                                                                      

In [20]:
medium_reasoner = MediumKGReasoner(kg_model, medium_reasoner_config)

In [21]:
for query in QUERY_EXAMPLES:
    print("\nQuery: ", query)
    answer, rinfo = medium_reasoner.perform(query)
    print('Return status: ')
    pprint(rinfo)
    print("Answer: ", answer)


Query:  Which of the following people walked along the highway: Sasha, Masha, Katya?
Return status: 
ReturnInfo(occurred_warning=[], status=<ReturnStatus.success: 0>, message='')
Answer:  <|NotEnoughtInfo|>

Query:  Have motorboat was ever sailed through a water canal?
Return status: 
ReturnInfo(occurred_warning=[], status=<ReturnStatus.success: 0>, message='')
Answer:  Based on the provided information, it is not possible to determine if a motorboat has ever been sailed through a water canal with certainty. However, the general guideline suggests that motorboats of certain dimensions (length up to 40-50 feet, width up to 12-15 feet, and height up to 10-12 feet) can be transported through canals.

However, it's worth noting that sailing a motorboat through a water canal is not typically how these vessels are transported. Motorboats are usually towed or carried on trailers through canals, rather than being sailed.

Therefore, the answer to the question is: 
"it is unlikely that a motor

In [23]:
print("qa tgen cache:")
pprint(weak_reasoner.get_agent_tgen_stat())
print("qa casual cache:")
pprint(weak_reasoner.get_cache_stat())

qa tgen cache:
{'answer_generator': {'answer_generator_solver': None},
 'query_parser': {'kw_extraction_solver': None}}
qa casual cache:
{'WeakKGReasoner': None,
 'answer_generator': {'QALLMGenerator': None, 'answer_generator_solver': None},
 'knowledge_comparator': {'KnowledgeComparator': None},
 'knowledge_retriever': {'KnowledgeRetriever': None,
                         'triplets_filter': {'TripletsFilter': None},
                         'triplets_retriever': {'BeamSearchTripletsRetriever': {'BeamSearchTripletsRetriever': None,
                                                                                'traversal_cache': None}}},
 'query_parser': {'QueryLLMParser': None, 'kw_extraction_solver': None}}


In [24]:
del mem_pipeline

In [25]:
del kg_model